In [1]:
import yaml
from rdkit import Chem
from rdkit.Chem.Descriptors import MolWt
from MDSetup.forcefield import forcefield
from MDSetup.tools.systemsetup import (
    get_system_volume,
    generate_initial_configuration
)

# System building for LAMMPS and GROMACS

This notebook shows the capibility of the MDSetup forcefield class to map the predefined force field format to arbitrary SMILES given molecules.

In [2]:
# Define molecules via SMILES
smiles = [
    "CCC",
    "C=C"
]

# Define self given names for each molecule
residues = [
    "propane",
    "ethene"
]

# Define system conditions

# Molecule numbers of each species
molecule_numbers = [ 100, 300 ]

# Get molar masses from rdkit
molar_masses = [
    MolWt(Chem.MolFromSmiles(smile)) for smile in smiles
]

# Define density
density = 550

## Build system for LAMMPS

### Write molecule and topology files

In [3]:
# Read in LAMMPS forcefield files
forcefield_paths = [ 
    "../../force-fields/TAMie_hydrocarbons_lammps.toml" 
]

# Define building folder
building_folder = "lammps"

# Initialize force field class
ff = forcefield( 
    smiles = smiles,
    force_field_paths = forcefield_paths,
    verbose=True
)

# Define path to molecule template file
molecule_template = "../../templates/lammps/template_lammps.mol"

# Write molecule files in LAMMPS compatible format
ff.write_molecule_files(
    molecule_template = molecule_template, 
    molecule_path = building_folder,
    residues = residues
)

# Define path to the topology (force field) file
topology_template = "../../templates/lammps/template_lammps.params"

# Read in default settings for force field file
default_ff = yaml.safe_load(open("lammps/defaults.yaml"))

# Write topology (force field) file
ff.write_topology_file(
    topology_template = topology_template, 
    topology_path = building_folder, 
    system_name = "propane_ethene",
    **default_ff["nonbonded"]
)

print(f"\n\nMolecule files:\n {', '.join(ff.molecule_files)}\n")
print(f"\nTopology file:\n {ff.topology_file}")

Force field provided for software 'lammps'

Matching atoms to force field for SMILES: CCC

Atom 0 (C) is matched to ff type: CH3_alkane
Atom 1 (C) is matched to ff type: CH2_alkane
Atom 2 (C) is matched to ff type: CH3_alkane
Atom 3 (H) is matched to ff type: carbon_h
Atom 4 (H) is matched to ff type: carbon_h
Atom 5 (H) is matched to ff type: carbon_h
Atom 6 (H) is matched to ff type: carbon_h
Atom 7 (H) is matched to ff type: carbon_h
Atom 8 (H) is matched to ff type: carbon_h
Atom 9 (H) is matched to ff type: carbon_h
Atom 10 (H) is matched to ff type: carbon_h

Final molecule after removing carbon h's

Atom 0 (C): CH3_alkane
Atom 1 (C): CH2_alkane
Atom 2 (C): CH3_alkane

Matching atoms to force field for SMILES: C=C

Atom 0 (C) is matched to ff type: CH2_alkene
Atom 1 (C) is matched to ff type: CH2_alkene
Atom 2 (H) is matched to ff type: carbon_h
Atom 3 (H) is matched to ff type: carbon_h
Atom 4 (H) is matched to ff type: carbon_h
Atom 5 (H) is matched to ff type: carbon_h

Final 

### Write lammps input for building the system

In [4]:
# Define box type
box_setup = dict(
  box_type = "cubic",
  z_x_relation = 1.0,
  z_y_relation = 1.0
)

# Define unit conversion if unit is not Angstrom
distance_conversion = 1

# Get box dimensions 
box_dim = get_system_volume(
    molar_masses=molar_masses,
    molecule_numbers=molecule_numbers,
    density=density,
    unit_conversion=distance_conversion,
    **box_setup,
)

# Define bash template to run building
build_template = "../../templates/bash/build_system_lmp_pbs.sh"

# Define lammps building template
building_template = "../../templates/lammps/template_lammps.build"

# Define molecule list 
molecules_list = [
    { "name": res,
      "number": num 
    } for res,num in zip(residues,molecule_numbers)
]

# Generate initial configuration (needs LAMMPS on the machine)
generate_initial_configuration(
    destination_folder = building_folder,
    build_template = build_template,
    software = ff.software,
    coordinate_paths = ff.molecule_files,
    molecules_list = molecules_list,
    box = box_dim,
    build_input_template = building_template,
    force_field_file = ff.topology_file
)


Build system locally! Wait until it is finished.

Build successful



'lammps/init_conf.data'

## Build system for GROMACS

### Write itp, gro, and topology files

In [4]:
# Read in LAMMPS forcefield files
forcefield_paths = [ 
    "../../force-fields/TAMie_hydrocarbons_gromacs.toml" 
]

# Define building folder
building_folder = "gromacs"

# Initialize force field class
ff = forcefield( 
    smiles = smiles,
    force_field_paths = forcefield_paths,
    verbose=True
)

# Define path to molecule template file
molecule_template = "../../templates/gromacs/template_gromacs.itp"
gro_template = "../../templates/gromacs/template_gromacs.gro"

# Define number of exlucded bonded interactions per molecule
# e.g.: 3 means that bond, angle, and dihedral interactions are excluded
nrexcl = [ 3, 3 ]

# Define molecule list 
residue_dict = { 
    res: num for res,num in zip(residues,molecule_numbers)
}

# Write molecule files in LAMMPS compatible format
ff.write_molecule_files(
    molecule_template = molecule_template, 
    molecule_path = building_folder,
    residues = residues,
    gro_template = gro_template,
    nrexcl = nrexcl
)

# Define path to the topology (force field) file
topology_template = "../../templates/gromacs/template_gromacs.top"

# Read in default settings for force field file
default_ff = yaml.safe_load(open("gromacs/defaults.yaml"))

# Write topology (force field) file
ff.write_topology_file(
    topology_template = topology_template, 
    topology_path = building_folder, 
    system_name = "propane_ethene",
    residue_dict = residue_dict,
    **default_ff["nonbonded"]
)

print(f"\n\nMolecule files:\n {', '.join(ff.molecule_files)}\n")
print(f"\nTopology file:\n {ff.topology_file}")

Force field provided for software 'gromacs'

Matching atoms to force field for SMILES: CCC

Atom 0 (C) is matched to ff type: CH3_alkane
Atom 1 (C) is matched to ff type: CH2_alkane
Atom 2 (C) is matched to ff type: CH3_alkane
Atom 3 (H) is matched to ff type: carbon_h
Atom 4 (H) is matched to ff type: carbon_h
Atom 5 (H) is matched to ff type: carbon_h
Atom 6 (H) is matched to ff type: carbon_h
Atom 7 (H) is matched to ff type: carbon_h
Atom 8 (H) is matched to ff type: carbon_h
Atom 9 (H) is matched to ff type: carbon_h
Atom 10 (H) is matched to ff type: carbon_h

Final molecule after removing carbon h's

Atom 0 (C): CH3_alkane
Atom 1 (C): CH2_alkane
Atom 2 (C): CH3_alkane

Matching atoms to force field for SMILES: C=C

Atom 0 (C) is matched to ff type: CH2_alkene
Atom 1 (C) is matched to ff type: CH2_alkene
Atom 2 (H) is matched to ff type: carbon_h
Atom 3 (H) is matched to ff type: carbon_h
Atom 4 (H) is matched to ff type: carbon_h
Atom 5 (H) is matched to ff type: carbon_h

Final

In [5]:
# Define box type
box_setup = dict(
  box_type = "cubic",
  z_x_relation = 1.0,
  z_y_relation = 1.0
)

# Define unit conversion if unit is not Angstrom
distance_conversion = 1/10

# Get box dimensions 
box_dim = get_system_volume(
    molar_masses=molar_masses,
    molecule_numbers=molecule_numbers,
    density=density,
    unit_conversion=distance_conversion,
    **box_setup,
)

# Define bash template to run building
build_template = "../../templates/bash/build_system_gmx_pbs.sh"

# Define molecule list 
molecules_list = [
    { "name": res,
      "number": num 
    } for res,num in zip(residues,molecule_numbers)
]

# Generate initial configuration (needs LAMMPS on the machine)
generate_initial_configuration(
    destination_folder = building_folder,
    build_template = build_template,
    software = ff.software,
    coordinate_paths = ff.molecule_files,
    molecules_list = molecules_list,
    box = box_dim
)


Build system locally! Wait until it is finished.

Build successful



'gromacs/init_conf.gro'